Chào bạn!

Rất vui khi thấy bạn đã tiến triển với dự án của mình. Dựa trên yêu cầu của bạn, bạn đang cân nhắc giữa hai phương pháp để xử lý hàng loạt video YouTube:

1. **Sử dụng API hiện tại để chạy hàng loạt video**: Gửi nhiều yêu cầu tới API của bạn để xử lý từng video một.
2. **Chuyển đổi API sang script để xử lý hàng loạt video cục bộ**: Viết một script Python riêng biệt để xử lý nhiều video cùng lúc mà không cần phải gửi nhiều yêu cầu HTTP tới API.

Dưới đây, tôi sẽ phân tích chi tiết cả hai phương pháp, ưu và nhược điểm của từng phương pháp, cũng như cung cấp hướng dẫn cụ thể để bạn có thể lựa chọn phương pháp phù hợp nhất với nhu cầu của mình.

---

## **Phương Pháp 1: Sử Dụng API Hiện Tại để Chạy Hàng Loạt Video**

### **Ưu Điểm:**

1. **Tính linh hoạt và mở rộng**: Bạn có thể dễ dàng tích hợp API này vào các ứng dụng web hoặc dịch vụ khác.
2. **Bảo mật**: API Key giúp bảo vệ tài nguyên và kiểm soát truy cập.
3. **Quản lý dễ dàng**: Bạn có thể triển khai API trên các nền tảng như Google Cloud Functions, AWS Lambda, Heroku, v.v., để dễ dàng mở rộng khi cần.

### **Nhược Điểm:**

1. **Tốc độ và hiệu suất**: Gửi nhiều yêu cầu HTTP có thể chậm và tăng độ trễ do giới hạn về tốc độ mạng và khả năng xử lý của server.
2. **Giới hạn về tỷ lệ yêu cầu**: Các nền tảng API thường có giới hạn về số lượng yêu cầu mỗi phút hoặc mỗi ngày, có thể gây ra lỗi nếu bạn gửi quá nhiều yêu cầu.
3. **Phức tạp trong việc quản lý đồng thời**: Nếu bạn cần xử lý hàng nghìn video, việc gửi nhiều yêu cầu đồng thời có thể gây ra các vấn đề về tài nguyên và quản lý.

### **Cách Triển Khai Hàng Loạt qua API:**

Để sử dụng API hiện tại của bạn để xử lý hàng loạt video, bạn có thể viết một script Python riêng biệt (tách biệt với API) để gửi nhiều yêu cầu tới API của bạn. Dưới đây là ví dụ về cách thực hiện điều này:

#### **1. Tạo File `batch_process.py`**

```python
import requests
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Địa chỉ URL của API của bạn
API_URL = "https://your-api-endpoint.com/get_youtube_transcript"

# API Key cho API của bạn
CLIENT_API_KEY = "yt2024_k8hj3n5m9p2q4w7r"

# Danh sách các URL YouTube bạn muốn xử lý
youtube_urls = [
    "https://www.youtube.com/watch?v=VIDEO_ID_1",
    "https://www.youtube.com/watch?v=VIDEO_ID_2",
    "https://www.youtube.com/watch?v=VIDEO_ID_3",
    # Thêm các URL khác ở đây
]

# Hàm gửi yêu cầu tới API
def process_video(url):
    headers = {
        'X-API-Key': CLIENT_API_KEY,
        'Content-Type': 'application/json'
    }
    params = {
        'youtube_url': url
    }
    try:
        response = requests.get(API_URL, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi xử lý video {url}: {str(e)}")
        return {"error": str(e), "url": url}

def main():
    results = []
    max_workers = 5  # Số lượng luồng song song

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(process_video, url): url for url in youtube_urls}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                results.append(data)
                print(f"Đã xử lý xong video: {url}")
            except Exception as e:
                print(f"Lỗi khi xử lý video {url}: {str(e)}")

    # Lưu kết quả vào file JSON
    with open('batch_results.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    print("Đã lưu kết quả vào 'batch_results.json'")

if __name__ == "__main__":
    main()
```

#### **2. Giải Thích Script:**

- **ThreadPoolExecutor**: Sử dụng để gửi nhiều yêu cầu song song, giúp tăng tốc quá trình xử lý.
- **max_workers**: Xác định số lượng luồng song song. Bạn có thể điều chỉnh số này dựa trên khả năng của API và tài nguyên hệ thống.
- **Lưu trữ kết quả**: Kết quả được lưu vào file `batch_results.json` để dễ dàng kiểm tra và xử lý sau này.

#### **3. Chạy Script:**

```bash
python batch_process.py
```

#### **4. Lưu Ý:**

- **Giới hạn Tỷ lệ Yêu cầu**: Đảm bảo rằng bạn không vượt quá giới hạn tỷ lệ yêu cầu của API để tránh bị chặn. Bạn có thể thêm `time.sleep()` vào hàm `process_video` nếu cần thiết.
  
- **Xử Lý Lỗi**: Script đã bao gồm cơ chế xử lý lỗi cơ bản, nhưng bạn có thể mở rộng thêm để xử lý các trường hợp phức tạp hơn.

---

## **Phương Pháp 2: Chuyển Đổi API Sang Script để Xử Lý Hàng Loạt Video Cục Bộ**

### **Ưu Điểm:**

1. **Tốc độ và hiệu suất**: Xử lý trực tiếp cục bộ có thể nhanh hơn do không phải qua nhiều bước trung gian như gửi yêu cầu HTTP.
2. **Không bị giới hạn tỷ lệ yêu cầu**: Bạn không phải lo lắng về giới hạn tỷ lệ yêu cầu của API.
3. **Dễ dàng quản lý và bảo trì**: Bạn có thể dễ dàng kiểm soát và tùy chỉnh quá trình xử lý hàng loạt theo nhu cầu.

### **Nhược Điểm:**

1. **Bảo mật**: API Key được lưu trữ cục bộ có thể gặp rủi ro về bảo mật nếu không được quản lý đúng cách.
2. **Tài nguyên hệ thống**: Xử lý hàng loạt video đòi hỏi tài nguyên hệ thống cao hơn, đặc biệt nếu bạn xử lý nhiều video cùng lúc.
3. **Khả năng mở rộng**: Nếu bạn cần xử lý hàng ngàn video thường xuyên, script cục bộ có thể không phù hợp như một API triển khai trên cloud.

### **Cách Triển Khai Hàng Loạt qua Script:**

Bạn có thể sử dụng script API hiện tại của mình và mở rộng nó để xử lý nhiều video cùng lúc. Dưới đây là hướng dẫn chi tiết.

#### **1. Tạo File `batch_script.py`**

```python
import requests
import json
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.parse import urlparse, parse_qs
import os
from dotenv import load_dotenv
import logging
import sys
import xml.etree.ElementTree as ET
from datetime import timedelta

# Tải biến môi trường từ file .env
load_dotenv()

# Thiết lập logging
logging.basicConfig(
    level=logging.INFO,  # Có thể thay đổi thành logging.DEBUG để xem chi tiết hơn
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

# Rapid API key từ environment variable 
RAPID_API_KEY = os.environ.get('RAPID_API_KEY')
if not RAPID_API_KEY:
    logger.error("RAPID_API_KEY not found in environment variables")
    raise ValueError("Missing RAPID_API_KEY in environment")

logger.info(f"RAPID_API_KEY loaded: {RAPID_API_KEY[:5]}...")  # Log 5 ký tự đầu để kiểm tra

# Hàm trích xuất video ID từ URL YouTube
def extract_video_id(youtube_url):
    """Extract video ID from YouTube URL"""
    try:
        # Thêm https:// nếu URL không có
        if not youtube_url.startswith(('http://', 'https://')):
            youtube_url = 'https://' + youtube_url
                
        parsed_url = urlparse(youtube_url)
            
        # Kiểm tra hostname hợp lệ
        if parsed_url.hostname in ['www.youtube.com', 'youtube.com']:
            if parsed_url.path == '/watch':
                return parse_qs(parsed_url.query)['v'][0]
            elif parsed_url.path.startswith('/embed/'):
                return parsed_url.path.split('/')[2]
            elif parsed_url.path.startswith('/v/'):
                return parsed_url.path.split('/')[2]
                    
        # Trường hợp URL ngắn youtu.be
        elif parsed_url.hostname == 'youtu.be':
            return parsed_url.path.lstrip('/')
                
        # Trường hợp URL sử dụng Handle
        elif parsed_url.path.startswith('/@'):
            handle = parsed_url.path.split('/')[1]
            return get_channel_id_from_handle(handle)
                
        return None
            
    except Exception as e:
        logger.error(f"Error extracting video ID: {str(e)}")
        return None

def get_channel_id_from_handle(handle):
    """
    Lấy Channel ID từ Handle bằng cách truy cập trang About và phân tích HTML.
    """
    try:
        about_url = f"https://www.youtube.com/{handle}/about"
        logger.info(f"Truy cập trang About: {about_url}")
        response = requests.get(about_url)
        response.raise_for_status()

        # Sử dụng BeautifulSoup để phân tích HTML
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        scripts = soup.find_all('script')

        for script in scripts:
            if 'channelId' in script.text:
                # Sử dụng regex để tìm Channel ID
                import re
                match = re.search(r'"channelId":"(UC[A-Za-z0-9_-]{22})"', script.text)
                if match:
                    channel_id = match.group(1)
                    logger.info(f"Channel ID tìm thấy: {channel_id}")
                    return channel_id

        logger.error(f"Không tìm thấy Channel ID trên trang About của handle: {handle}")
        return None

    except Exception as e:
        logger.error(f"Lỗi trong hàm get_channel_id_from_handle: {str(e)}")
        return None

def get_video_details(video_id):
    """Get video details from API"""
    url = "https://youtube-media-downloader.p.rapidapi.com/v2/video/details"
    
    headers = {
        "X-RapidAPI-Key": RAPID_API_KEY,
        "X-RapidAPI-Host": "youtube-media-downloader.p.rapidapi.com"
    }
    
    params = {"videoId": video_id}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logger.error(f"API call error for video {video_id}: {str(e)}")
        return None

def get_transcript_from_subtitles(subtitle_url, max_retries=3):
    """Get and process transcript from subtitles URL with retries"""
    for attempt in range(max_retries):
        try:
            transcript_lines = []
            subtitle_response = requests.get(subtitle_url)
            
            if subtitle_response.status_code == 200:
                content = subtitle_response.content
                if not content or len(content.strip()) == 0:
                    raise ValueError(f"Empty response content for URL: {subtitle_url}")
                
                root = ET.fromstring(content)
                for elem in root.iter('text'):
                    start = elem.get('start')
                    text = elem.text
                    if start and text:
                        start_seconds = float(start)
                        start_time = str(timedelta(seconds=start_seconds))
                        hh_mm_ss = start_time.split('.')[0]
                        transcript_lines.append(f"[{hh_mm_ss}] {text}")
                
                if transcript_lines:
                    transcript_text = '\n'.join(transcript_lines)
                    if len(transcript_text) > 95000:
                        transcript_text = transcript_text[:95000] + "\n[Transcript bị cắt do quá dài]"
                    return transcript_text
                    
            else:
                logger.warning(f"Failed to retrieve subtitle data: {subtitle_response.status_code}")
                if attempt < max_retries - 1:
                    time.sleep(2)
                continue
                
        except (ET.ParseError, ValueError, requests.exceptions.RequestException) as e:
            logger.warning(f"Error processing subtitles (attempt {attempt + 1}/{max_retries}): {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(2)
            continue
            
    return ""

def get_video_comments(video_id):
    """Get comments for a YouTube video"""
    try:
        # Tạo connection
        conn = http.client.HTTPSConnection("youtube-media-downloader.p.rapidapi.com")
        
        # Headers chính xác như mẫu
        headers = {
            'X-RapidAPI-Key': RAPID_API_KEY,
            'X-RapidAPI-Host': "youtube-media-downloader.p.rapidapi.com"
        }
        
        all_comments = []
        formatted_comments = []
        next_token = None
        
        while True:
            # Build URL
            url = f"/v2/video/comments?videoId={video_id}&sortBy=top"
            if next_token:
                url += f"&nextToken={next_token}"
                
            # Make request
            conn.request("GET", url, headers=headers)
            response = conn.getresponse()
            data = json.loads(response.read().decode("utf-8"))
            
            if not data.get("status"):
                error_id = data.get("errorId", "Unknown error")
                if error_id == "VideoNotFoundOrCommentDisabled":
                    return {
                        "count": 0,
                        "comments": [],
                        "status": "DISABLED: Comments are disabled or video not found"
                    }
                else:
                    raise Exception(error_id)
            
            # Get comments from response
            comments = data.get("items", [])
            
            # Format comments
            for comment in comments:
                user_name = comment.get('channel', {}).get('name', '')
                text = comment.get('contentText', '')
                formatted_comment = f"[{user_name}]\n[{text}]"
                formatted_comments.append(formatted_comment)
            
            # Check for next page
            next_token = data.get("nextToken")
            if not next_token:
                break
                
        # Split comments if needed
        if formatted_comments:
            comment_parts = split_comments(formatted_comments)
            return {
                "count": len(formatted_comments),
                "comments": comment_parts,
                "status": f"SUCCESS: {len(formatted_comments)} comments saved in {len(comment_parts)} parts"
            }
        else:
            return {
                "count": 0,
                "comments": [],
                "status": "NO_COMMENTS: No comments found"
            }
            
    except Exception as e:
        logger.error(f"Error getting comments for video {video_id}: {str(e)}")
        return {
            "count": 0,
            "comments": [],
            "status": f"ERROR: {str(e)}"
        }

def split_comments(formatted_comments, max_chars=100000):
    """Split comments list into smaller parts under max_chars"""
    parts = []
    current_part = []
    current_length = 0
    
    for comment in formatted_comments:
        comment_length = len(comment) + 2  # Add 2 for \n\n between comments
        
        if current_length + comment_length > max_chars and current_part:
            parts.append("\n\n".join(current_part))
            current_part = [comment]
            current_length = comment_length
        else:
            current_part.append(comment)
            current_length += comment_length
    
    if current_part:
        parts.append("\n\n".join(current_part))
    
    return parts

def convert_subscriber_count(count_text):
    """Convert subscriber count from text (e.g. '2.19M subscribers') to number"""
    try:
        if not count_text:
            return 0
        count = count_text.replace('subscribers', '').strip()
        multiplier = 1
        if 'K' in count:
            multiplier = 1000
            count = count.replace('K', '')
        elif 'M' in count:
            multiplier = 1000000
            count = count.replace('M', '')
        elif 'B' in count:
            multiplier = 1000000000
            count = count.replace('B', '')
        return int(float(count) * multiplier)
    except (ValueError, AttributeError):
        logger.warning(f"Error converting subscriber count: {count_text}")
        return 0

def process_video_data(video_data):
    """Process and format video data"""
    # Get subtitles URL safely
    subtitles_items = video_data.get("subtitles", {}).get("items", [])
    subtitles_url = subtitles_items[0].get("url", "") if subtitles_items else ""
    
    # Get transcript if subtitles URL exists
    transcript = ""
    if subtitles_url:
        transcript = get_transcript_from_subtitles(subtitles_url)
    
    # Convert subscriber count
    subscriber_count_text = video_data.get("channel", {}).get("subscriberCountText", "")
    subscriber_count = convert_subscriber_count(subscriber_count_text)
    
    # Format is_live to string
    is_live = "true" if video_data.get("isLive", False) else "false"
    
    # Get video ID for comments
    video_id = video_data.get('id', '')
    comments_data = get_video_comments(video_id)
    
    response = {
        "title": video_data.get("title", ""),
        "description": video_data.get("description", ""),
        "username": video_data.get("channel", {}).get("handle", ""),
        "user_screen_name": video_data.get("channel", {}).get("name", ""),
        "user_id": video_data.get("channel", {}).get("id", ""),
        "user_subscribers": subscriber_count,
        "published_time": video_data.get("publishedTime", ""),
        "viewCount": video_data.get("viewCount", 0),
        "likeCount": video_data.get("likeCount", 0),
        "subtitles_url": subtitles_url,
        "duration": video_data.get("duration", ""),
        "thumbnail_url": video_data.get("thumbnail", [{}])[0].get("url", ""),
        "is_live": is_live,
        "category": video_data.get("category", ""),
        "transcript": transcript,
        "url": f"https://youtube.com/watch?v={video_data.get('id', '')}",
        "comments_count": comments_data["count"],
        "comments_status": comments_data["status"]
    }
    
    # Add comments parts if they exist
    if comments_data["comments"]:
        for i, part in enumerate(comments_data["comments"], 1):
            response[f"comments_{i}"] = part
            
    return response

def process_single_video(youtube_url):
    """Process a single YouTube video and return formatted data"""
    video_id = extract_video_id(youtube_url)
    if not video_id:
        logger.error(f"Invalid YouTube URL: {youtube_url}")
        return {"error": "Invalid YouTube URL", "url": youtube_url}
    
    video_data = get_video_details(video_id)
    if not video_data:
        logger.error(f"Failed to fetch video details for video ID: {video_id}")
        return {"error": "Failed to fetch video details", "url": youtube_url}
    
    formatted_data = process_video_data(video_data)
    formatted_data["url_provided"] = youtube_url
    return formatted_data

def main():
    """Main function to process a list of YouTube URLs"""
    # Danh sách các URL YouTube bạn muốn xử lý
    youtube_urls = [
        "https://www.youtube.com/watch?v=VIDEO_ID_1",
        "https://www.youtube.com/watch?v=VIDEO_ID_2",
        "https://youtu.be/VIDEO_ID_3",
        "https://www.youtube.com/@moxierobot",
        # Thêm các URL khác ở đây
    ]
    
    results = []
    max_workers = 5  # Số lượng luồng song song

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(process_single_video, url): url for url in youtube_urls}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                results.append(data)
                logger.info(f"Đã xử lý xong video: {url}")
            except Exception as e:
                logger.error(f"Lỗi khi xử lý video {url}: {str(e)}")
                results.append({"error": str(e), "url": url})

    # Lưu kết quả vào file JSON
    with open('batch_results.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    logger.info("Đã lưu kết quả vào 'batch_results.json'")

if __name__ == "__main__":
    main()
```

#### **2. Giải Thích Script:**

- **ThreadPoolExecutor**: Sử dụng để xử lý nhiều video song song, giúp tăng tốc quá trình xử lý.
  
- **Danh sách URL YouTube**: Bạn có thể thêm nhiều URL YouTube vào danh sách `youtube_urls` để xử lý hàng loạt.

- **Xử lý Handle**: Hàm `extract_video_id` đã được mở rộng để xử lý các URL sử dụng Handle (ví dụ: `https://www.youtube.com/@moxierobot`).

- **Xử lý Transcript và Comments**: Script sẽ lấy transcript và comments cho từng video, xử lý và lưu trữ chúng.

- **Lưu kết quả**: Kết quả được lưu vào file `batch_results.json`.

#### **3. Chạy Script:**

```bash
python batch_script.py
```

#### **4. Lưu Ý:**

- **API Key Bảo Mật**: Đảm bảo rằng file `.env` chứa biến `RAPID_API_KEY` không được đẩy lên các kho mã nguồn công khai.

- **Giới hạn Tỷ lệ Yêu cầu**: Mặc dù bạn không phải lo lắng về giới hạn tỷ lệ của API như khi sử dụng API từ bên ngoài, nhưng vẫn nên kiểm soát số lượng yêu cầu song song để tránh quá tải hệ thống hoặc API.

- **Xử lý Lỗi**: Script đã bao gồm cơ chế xử lý lỗi cơ bản, nhưng bạn có thể mở rộng thêm để xử lý các trường hợp phức tạp hơn.

---

## **So Sánh Hai Phương Pháp**

| Tiêu chí                  | Sử dụng API Hiện Tại                  | Chuyển Đổi Sang Script Cục Bộ          |
|---------------------------|---------------------------------------|----------------------------------------|
| **Tốc độ và Hiệu suất**   | Trung bình (tuỳ thuộc vào số lượng yêu cầu và tốc độ mạng) | Cao hơn (xử lý trực tiếp cục bộ)        |
| **Quản lý Tỷ lệ Yêu cầu** | Phụ thuộc vào giới hạn API             | Không bị giới hạn bởi API               |
| **Bảo mật**               | Tốt (sử dụng API Key)                  | Kém hơn nếu không quản lý API Key tốt   |
| **Khả năng Mở rộng**      | Cao (dễ dàng triển khai trên cloud)    | Trung bình (phụ thuộc vào tài nguyên hệ thống) |
| **Độ phức tạp**           | Phức tạp hơn trong việc gửi nhiều yêu cầu | Đơn giản hơn khi xử lý cục bộ          |
| **Tính linh hoạt**        | Cao (dễ dàng tích hợp vào các ứng dụng khác) | Trung bình (chỉ xử lý cục bộ)           |

---

## **Kết Luận và Khuyến Nghị**

- **Nếu bạn cần tích hợp xử lý video vào các ứng dụng web hoặc dịch vụ khác**, việc sử dụng API hiện tại để xử lý hàng loạt thông qua nhiều yêu cầu song song là phù hợp hơn. Bạn có thể tối ưu hóa script `batch_process.py` để gửi nhiều yêu cầu một cách hiệu quả.

- **Nếu bạn chủ yếu xử lý video hàng loạt một cách cục bộ**, việc chuyển đổi API thành một script riêng biệt như `batch_script.py` sẽ mang lại hiệu suất cao hơn và quản lý dễ dàng hơn.

Dưới đây là một số gợi ý bổ sung để tối ưu hóa quá trình xử lý:

### **1. Sử Dụng Asynchronous Requests**

Thay vì sử dụng `ThreadPoolExecutor`, bạn có thể sử dụng các thư viện như `aiohttp` và `asyncio` để xử lý các yêu cầu một cách không đồng bộ, giúp tăng hiệu suất hơn nữa.

#### **Ví Dụ với `aiohttp`:**

```python
import asyncio
import aiohttp
import json
from urllib.parse import urlparse, parse_qs
import os
from dotenv import load_dotenv
import logging
import sys
import xml.etree.ElementTree as ET
from datetime import timedelta

# Tải biến môi trường từ file .env
load_dotenv()

# Thiết lập logging
logging.basicConfig(
    level=logging.INFO,  # Có thể thay đổi thành logging.DEBUG để xem chi tiết hơn
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

# Rapid API key từ environment variable 
RAPID_API_KEY = os.environ.get('RAPID_API_KEY')
if not RAPID_API_KEY:
    logger.error("RAPID_API_KEY not found in environment variables")
    raise ValueError("Missing RAPID_API_KEY in environment")

logger.info(f"RAPID_API_KEY loaded: {RAPID_API_KEY[:5]}...")  # Log 5 ký tự đầu để kiểm tra

async def fetch(session, url, headers=None, params=None):
    """Fetch data từ một URL sử dụng aiohttp"""
    async with session.get(url, headers=headers, params=params) as response:
        response.raise_for_status()
        return await response.json()

async def process_video(session, youtube_url):
    """Xử lý một video YouTube"""
    video_id = extract_video_id(youtube_url)
    if not video_id:
        logger.error(f"Invalid YouTube URL: {youtube_url}")
        return {"error": "Invalid YouTube URL", "url": youtube_url}
    
    # Get video details
    try:
        video_data = await get_video_details_async(session, video_id)
        if not video_data:
            logger.error(f"Failed to fetch video details for video ID: {video_id}")
            return {"error": "Failed to fetch video details", "url": youtube_url}
        
        # Process video data
        formatted_data = process_video_data(video_data)
        formatted_data["url_provided"] = youtube_url
        return formatted_data
    except Exception as e:
        logger.error(f"Error processing video {youtube_url}: {str(e)}")
        return {"error": str(e), "url": youtube_url}

def extract_video_id(youtube_url):
    """Extract video ID từ URL YouTube"""
    try:
        # Thêm https:// nếu URL không có
        if not youtube_url.startswith(('http://', 'https://')):
            youtube_url = 'https://' + youtube_url
                
        parsed_url = urlparse(youtube_url)
            
        # Kiểm tra hostname hợp lệ
        if parsed_url.hostname in ['www.youtube.com', 'youtube.com']:
            if parsed_url.path == '/watch':
                return parse_qs(parsed_url.query)['v'][0]
            elif parsed_url.path.startswith('/embed/'):
                return parsed_url.path.split('/')[2]
            elif parsed_url.path.startswith('/v/'):
                return parsed_url.path.split('/')[2]
                    
        # Trường hợp URL ngắn youtu.be
        elif parsed_url.hostname == 'youtu.be':
            return parsed_url.path.lstrip('/')
                
        # Trường hợp URL sử dụng Handle
        elif parsed_url.path.startswith('/@'):
            handle = parsed_url.path.split('/')[1]
            return get_channel_id_from_handle(handle)
                
        return None
            
    except Exception as e:
        logger.error(f"Error extracting video ID: {str(e)}")
        return None

def get_channel_id_from_handle(handle):
    """
    Lấy Channel ID từ Handle bằng cách truy cập trang About và phân tích HTML.
    """
    try:
        about_url = f"https://www.youtube.com/{handle}/about"
        logger.info(f"Truy cập trang About: {about_url}")
        response = requests.get(about_url)
        response.raise_for_status()

        # Sử dụng BeautifulSoup để phân tích HTML
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        scripts = soup.find_all('script')

        for script in scripts:
            if 'channelId' in script.text:
                # Sử dụng regex để tìm Channel ID
                import re
                match = re.search(r'"channelId":"(UC[A-Za-z0-9_-]{22})"', script.text)
                if match:
                    channel_id = match.group(1)
                    logger.info(f"Channel ID tìm thấy: {channel_id}")
                    return channel_id

        logger.error(f"Không tìm thấy Channel ID trên trang About của handle: {handle}")
        return None

    except Exception as e:
        logger.error(f"Lỗi trong hàm get_channel_id_from_handle: {str(e)}")
        return None

async def get_video_details_async(session, video_id):
    """Get video details từ API sử dụng aiohttp"""
    url = "https://youtube-media-downloader.p.rapidapi.com/v2/video/details"
    
    headers = {
        "X-RapidAPI-Key": RAPID_API_KEY,
        "X-RapidAPI-Host": "youtube-media-downloader.p.rapidapi.com"
    }
    
    params = {"videoId": video_id}
    
    try:
        video_data = await fetch(session, url, headers=headers, params=params)
        return video_data
    except Exception as e:
        logger.error(f"API call error for video {video_id}: {str(e)}")
        return None

def convert_subscriber_count(count_text):
    """Convert subscriber count từ text (ví dụ: '2.19M subscribers') thành số"""
    try:
        if not count_text:
            return 0
        count = count_text.replace('subscribers', '').strip()
        multiplier = 1
        if 'K' in count:
            multiplier = 1000
            count = count.replace('K', '')
        elif 'M' in count:
            multiplier = 1000000
            count = count.replace('M', '')
        elif 'B' in count:
            multiplier = 1000000000
            count = count.replace('B', '')
        return int(float(count) * multiplier)
    except (ValueError, AttributeError):
        logger.warning(f"Error converting subscriber count: {count_text}")
        return 0

def get_transcript_from_subtitles(subtitle_url, max_retries=3):
    """Get và xử lý transcript từ URL subtitles với retries"""
    for attempt in range(max_retries):
        try:
            transcript_lines = []
            subtitle_response = requests.get(subtitle_url)
            
            if subtitle_response.status_code == 200:
                content = subtitle_response.content
                if not content or len(content.strip()) == 0:
                    raise ValueError(f"Empty response content for URL: {subtitle_url}")
                
                root = ET.fromstring(content)
                for elem in root.iter('text'):
                    start = elem.get('start')
                    text = elem.text
                    if start and text:
                        start_seconds = float(start)
                        start_time = str(timedelta(seconds=start_seconds))
                        hh_mm_ss = start_time.split('.')[0]
                        transcript_lines.append(f"[{hh_mm_ss}] {text}")
                
                if transcript_lines:
                    transcript_text = '\n'.join(transcript_lines)
                    if len(transcript_text) > 95000:
                        transcript_text = transcript_text[:95000] + "\n[Transcript bị cắt do quá dài]"
                    return transcript_text
                    
            else:
                logger.warning(f"Failed to retrieve subtitle data: {subtitle_response.status_code}")
                if attempt < max_retries - 1:
                    time.sleep(2)
                continue
                
        except (ET.ParseError, ValueError, requests.exceptions.RequestException) as e:
            logger.warning(f"Error processing subtitles (attempt {attempt + 1}/{max_retries}): {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(2)
            continue
            
    return ""

def get_video_comments(video_id):
    """Get comments cho một video YouTube"""
    try:
        # Tạo connection
        conn = http.client.HTTPSConnection("youtube-media-downloader.p.rapidapi.com")
        
        # Headers chính xác như mẫu
        headers = {
            'X-RapidAPI-Key': RAPID_API_KEY,
            'X-RapidAPI-Host': "youtube-media-downloader.p.rapidapi.com"
        }
        
        all_comments = []
        formatted_comments = []
        next_token = None
        
        while True:
            # Build URL
            url = f"/v2/video/comments?videoId={video_id}&sortBy=top"
            if next_token:
                url += f"&nextToken={next_token}"
                
            # Make request
            conn.request("GET", url, headers=headers)
            response = conn.getresponse()
            data = json.loads(response.read().decode("utf-8"))
            
            if not data.get("status"):
                error_id = data.get("errorId", "Unknown error")
                if error_id == "VideoNotFoundOrCommentDisabled":
                    return {
                        "count": 0,
                        "comments": [],
                        "status": "DISABLED: Comments are disabled or video not found"
                    }
                else:
                    raise Exception(error_id)
            
            # Get comments from response
            comments = data.get("items", [])
            
            # Format comments
            for comment in comments:
                user_name = comment.get('channel', {}).get('name', '')
                text = comment.get('contentText', '')
                formatted_comment = f"[{user_name}]\n[{text}]"
                formatted_comments.append(formatted_comment)
            
            # Check for next page
            next_token = data.get("nextToken")
            if not next_token:
                break
                
        # Split comments if needed
        if formatted_comments:
            comment_parts = split_comments(formatted_comments)
            return {
                "count": len(formatted_comments),
                "comments": comment_parts,
                "status": f"SUCCESS: {len(formatted_comments)} comments saved in {len(comment_parts)} parts"
            }
        else:
            return {
                "count": 0,
                "comments": [],
                "status": "NO_COMMENTS: No comments found"
            }
            
    except Exception as e:
        logger.error(f"Error getting comments for video {video_id}: {str(e)}")
        return {
            "count": 0,
            "comments": [],
            "status": f"ERROR: {str(e)}"
        }

def split_comments(formatted_comments, max_chars=100000):
    """Split comments list into smaller parts dưới max_chars"""
    parts = []
    current_part = []
    current_length = 0
    
    for comment in formatted_comments:
        comment_length = len(comment) + 2  # Thêm 2 cho \n\n giữa các comments
        
        if current_length + comment_length > max_chars and current_part:
            parts.append("\n\n".join(current_part))
            current_part = [comment]
            current_length = comment_length
        else:
            current_part.append(comment)
            current_length += comment_length
    
    if current_part:
        parts.append("\n\n".join(current_part))
    
    return parts

def process_video_data(video_data):
    """Process và format video data"""
    # Get subtitles URL safely
    subtitles_items = video_data.get("subtitles", {}).get("items", [])
    subtitles_url = subtitles_items[0].get("url", "") if subtitles_items else ""
    
    # Get transcript nếu có subtitles URL
    transcript = ""
    if subtitles_url:
        transcript = get_transcript_from_subtitles(subtitles_url)
    
    # Convert subscriber count
    subscriber_count_text = video_data.get("channel", {}).get("subscriberCountText", "")
    subscriber_count = convert_subscriber_count(subscriber_count_text)
    
    # Format is_live to string
    is_live = "true" if video_data.get("isLive", False) else "false"
    
    # Get video ID for comments
    video_id = video_data.get('id', '')
    comments_data = get_video_comments(video_id)
    
    response = {
        "title": video_data.get("title", ""),
        "description": video_data.get("description", ""),
        "username": video_data.get("channel", {}).get("handle", ""),
        "user_screen_name": video_data.get("channel", {}).get("name", ""),
        "user_id": video_data.get("channel", {}).get("id", ""),
        "user_subscribers": subscriber_count,
        "published_time": video_data.get("publishedTime", ""),
        "viewCount": video_data.get("viewCount", 0),
        "likeCount": video_data.get("likeCount", 0),
        "subtitles_url": subtitles_url,
        "duration": video_data.get("duration", ""),
        "thumbnail_url": video_data.get("thumbnail", [{}])[0].get("url", ""),
        "is_live": is_live,
        "category": video_data.get("category", ""),
        "transcript": transcript,
        "url": f"https://youtube.com/watch?v={video_data.get('id', '')}",
        "comments_count": comments_data["count"],
        "comments_status": comments_data["status"]
    }
    
    # Thêm comments parts nếu có
    if comments_data["comments"]:
        for i, part in enumerate(comments_data["comments"], 1):
            response[f"comments_{i}"] = part
            
    return response

async def get_transcript_from_subtitles_async(session, subtitle_url, max_retries=3):
    """Get và xử lý transcript từ subtitles URL với retries (async)"""
    for attempt in range(max_retries):
        try:
            transcript_lines = []
            async with session.get(subtitle_url) as subtitle_response:
                if subtitle_response.status == 200:
                    content = await subtitle_response.text()
                    if not content or len(content.strip()) == 0:
                        raise ValueError(f"Empty response content for URL: {subtitle_url}")
                    
                    root = ET.fromstring(content)
                    for elem in root.iter('text'):
                        start = elem.get('start')
                        text = elem.text
                        if start and text:
                            start_seconds = float(start)
                            start_time = str(timedelta(seconds=start_seconds))
                            hh_mm_ss = start_time.split('.')[0]
                            transcript_lines.append(f"[{hh_mm_ss}] {text}")
                    
                    if transcript_lines:
                        transcript_text = '\n'.join(transcript_lines)
                        if len(transcript_text) > 95000:
                            transcript_text = transcript_text[:95000] + "\n[Transcript bị cắt do quá dài]"
                        return transcript_text
                    
                else:
                    logger.warning(f"Failed to retrieve subtitle data: {subtitle_response.status}")
                    if attempt < max_retries - 1:
                        await asyncio.sleep(2)
                    continue
                    
        except (ET.ParseError, ValueError, aiohttp.ClientError) as e:
            logger.warning(f"Error processing subtitles (attempt {attempt + 1}/{max_retries}): {str(e)}")
            if attempt < max_retries - 1:
                await asyncio.sleep(2)
            continue
                
    return ""

def convert_subscriber_count(count_text):
    """Convert subscriber count từ text (ví dụ: '2.19M subscribers') thành số"""
    try:
        if not count_text:
            return 0
        count = count_text.replace('subscribers', '').strip()
        multiplier = 1
        if 'K' in count:
            multiplier = 1000
            count = count.replace('K', '')
        elif 'M' in count:
            multiplier = 1000000
            count = count.replace('M', '')
        elif 'B' in count:
            multiplier = 1000000000
            count = count.replace('B', '')
        return int(float(count) * multiplier)
    except (ValueError, AttributeError):
        logger.warning(f"Error converting subscriber count: {count_text}")
        return 0

async def get_video_comments_async(session, video_id):
    """Get comments cho một video YouTube sử dụng aiohttp"""
    try:
        url = f"https://youtube-media-downloader.p.rapidapi.com/v2/video/comments?videoId={video_id}&sortBy=top"
        headers = {
            'X-RapidAPI-Key': RAPID_API_KEY,
            'X-RapidAPI-Host': "youtube-media-downloader.p.rapidapi.com"
        }
        
        all_comments = []
        formatted_comments = []
        next_token = None
        
        while True:
            # Thêm nextToken nếu có
            params = {}
            if next_token:
                params["nextToken"] = next_token
                
            # Gửi yêu cầu
            data = await fetch(session, url, headers=headers, params=params)
            
            if not data.get("status"):
                error_id = data.get("errorId", "Unknown error")
                if error_id == "VideoNotFoundOrCommentDisabled":
                    return {
                        "count": 0,
                        "comments": [],
                        "status": "DISABLED: Comments are disabled or video not found"
                    }
                else:
                    raise Exception(error_id)
            
            # Lấy comments từ phản hồi
            comments = data.get("items", [])
            
            # Định dạng comments
            for comment in comments:
                user_name = comment.get('channel', {}).get('name', '')
                text = comment.get('contentText', '')
                formatted_comment = f"[{user_name}]\n[{text}]"
                formatted_comments.append(formatted_comment)
            
            # Kiểm tra next page
            next_token = data.get("nextToken")
            if not next_token:
                break
                
        # Split comments nếu cần
        if formatted_comments:
            comment_parts = split_comments(formatted_comments)
            return {
                "count": len(formatted_comments),
                "comments": comment_parts,
                "status": f"SUCCESS: {len(formatted_comments)} comments saved in {len(comment_parts)} parts"
            }
        else:
            return {
                "count": 0,
                "comments": [],
                "status": "NO_COMMENTS: No comments found"
            }
            
    except Exception as e:
        logger.error(f"Error getting comments for video {video_id}: {str(e)}")
        return {
            "count": 0,
            "comments": [],
            "status": f"ERROR: {str(e)}"
        }

def split_comments(formatted_comments, max_chars=100000):
    """Split comments list into smaller parts dưới max_chars"""
    parts = []
    current_part = []
    current_length = 0
    
    for comment in formatted_comments:
        comment_length = len(comment) + 2  # Thêm 2 cho \n\n giữa các comments
        
        if current_length + comment_length > max_chars and current_part:
            parts.append("\n\n".join(current_part))
            current_part = [comment]
            current_length = comment_length
        else:
            current_part.append(comment)
            current_length += comment_length
    
    if current_part:
        parts.append("\n\n".join(current_part))
    
    return parts

def get_transcript_from_subtitles(subtitle_url, max_retries=3):
    """Get và xử lý transcript từ subtitles URL với retries"""
    for attempt in range(max_retries):
        try:
            transcript_lines = []
            subtitle_response = requests.get(subtitle_url)
            
            if subtitle_response.status_code == 200:
                content = subtitle_response.content
                if not content or len(content.strip()) == 0:
                    raise ValueError(f"Empty response content for URL: {subtitle_url}")
                
                root = ET.fromstring(content)
                for elem in root.iter('text'):
                    start = elem.get('start')
                    text = elem.text
                    if start and text:
                        start_seconds = float(start)
                        start_time = str(timedelta(seconds=start_seconds))
                        hh_mm_ss = start_time.split('.')[0]
                        transcript_lines.append(f"[{hh_mm_ss}] {text}")
                
                if transcript_lines:
                    transcript_text = '\n'.join(transcript_lines)
                    if len(transcript_text) > 95000:
                        transcript_text = transcript_text[:95000] + "\n[Transcript bị cắt do quá dài]"
                    return transcript_text
                    
            else:
                logger.warning(f"Failed to retrieve subtitle data: {subtitle_response.status_code}")
                if attempt < max_retries - 1:
                    time.sleep(2)
                continue
                
        except (ET.ParseError, ValueError, requests.exceptions.RequestException) as e:
            logger.warning(f"Error processing subtitles (attempt {attempt + 1}/{max_retries}): {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(2)
            continue
            
    return ""

def convert_subscriber_count(count_text):
    """Convert subscriber count từ text (ví dụ: '2.19M subscribers') thành số"""
    try:
        if not count_text:
            return 0
        count = count_text.replace('subscribers', '').strip()
        multiplier = 1
        if 'K' in count:
            multiplier = 1000
            count = count.replace('K', '')
        elif 'M' in count:
            multiplier = 1000000
            count = count.replace('M', '')
        elif 'B' in count:
            multiplier = 1000000000
            count = count.replace('B', '')
        return int(float(count) * multiplier)
    except (ValueError, AttributeError):
        logger.warning(f"Error converting subscriber count: {count_text}")
        return 0

def process_video_data(video_data):
    """Process và format video data"""
    # Get subtitles URL safely
    subtitles_items = video_data.get("subtitles", {}).get("items", [])
    subtitles_url = subtitles_items[0].get("url", "") if subtitles_items else ""
    
    # Get transcript nếu có subtitles URL
    transcript = ""
    if subtitles_url:
        transcript = get_transcript_from_subtitles(subtitles_url)
    
    # Convert subscriber count
    subscriber_count_text = video_data.get("channel", {}).get("subscriberCountText", "")
    subscriber_count = convert_subscriber_count(subscriber_count_text)
    
    # Format is_live to string
    is_live = "true" if video_data.get("isLive", False) else "false"
    
    # Get video ID for comments
    video_id = video_data.get('id', '')
    comments_data = get_video_comments(video_id)
    
    response = {
        "title": video_data.get("title", ""),
        "description": video_data.get("description", ""),
        "username": video_data.get("channel", {}).get("handle", ""),
        "user_screen_name": video_data.get("channel", {}).get("name", ""),
        "user_id": video_data.get("channel", {}).get("id", ""),
        "user_subscribers": subscriber_count,
        "published_time": video_data.get("publishedTime", ""),
        "viewCount": video_data.get("viewCount", 0),
        "likeCount": video_data.get("likeCount", 0),
        "subtitles_url": subtitles_url,
        "duration": video_data.get("duration", ""),
        "thumbnail_url": video_data.get("thumbnail", [{}])[0].get("url", ""),
        "is_live": is_live,
        "category": video_data.get("category", ""),
        "transcript": transcript,
        "url": f"https://youtube.com/watch?v={video_data.get('id', '')}",
        "comments_count": comments_data["count"],
        "comments_status": comments_data["status"]
    }
    
    # Thêm comments parts nếu có
    if comments_data["comments"]:
        for i, part in enumerate(comments_data["comments"], 1):
            response[f"comments_{i}"] = part
            
    return response

async def get_video_details_async(session, video_id):
    """Get video details từ API sử dụng aiohttp"""
    url = "https://youtube-media-downloader.p.rapidapi.com/v2/video/details"
    
    headers = {
        "X-RapidAPI-Key": RAPID_API_KEY,
        "X-RapidAPI-Host": "youtube-media-downloader.p.rapidapi.com"
    }
    
    params = {"videoId": video_id}
    
    try:
        video_data = await fetch(session, url, headers=headers, params=params)
        return video_data
    except Exception as e:
        logger.error(f"API call error for video {video_id}: {str(e)}")
        return None

async def main():
    """Main function to process a list of YouTube URLs"""
    # Danh sách các URL YouTube bạn muốn xử lý
    youtube_urls = [
        "https://www.youtube.com/watch?v=VIDEO_ID_1",
        "https://www.youtube.com/watch?v=VIDEO_ID_2",
        "https://youtu.be/VIDEO_ID_3",
        "https://www.youtube.com/@moxierobot",
        # Thêm các URL khác ở đây
    ]
    
    results = []
    max_workers = 10  # Số lượng kết nối đồng thời

    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in youtube_urls:
            tasks.append(process_video(session, url))
        
        # Chạy các task không đồng bộ
        for future in asyncio.as_completed(tasks):
            result = await future
            results.append(result)
            logger.info(f"Processed video: {result.get('url_provided', 'Unknown')}")
    
    # Lưu kết quả vào file JSON
    with open('batch_results_async.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
    logger.info("Đã lưu kết quả vào 'batch_results_async.json'")

if __name__ == "__main__":
    asyncio.run(main())
```

#### **2. Giải Thích Script:**

- **Asynchronous Requests (`aiohttp` và `asyncio`)**: Sử dụng để gửi nhiều yêu cầu một cách không đồng bộ, giúp tăng hiệu suất xử lý hàng loạt.

- **Hàm `fetch`**: Truy vấn bất kỳ URL nào và trả về dữ liệu JSON.

- **Hàm `process_video`**: Xử lý một video YouTube, bao gồm trích xuất ID, lấy chi tiết video, transcript và comments.

- **Hàm `main`**: Tạo các task để xử lý các video cùng lúc và thu thập kết quả.

#### **3. Cài Đặt Thư Viện Cần Thiết:**

Đảm bảo rằng bạn đã cài đặt các thư viện cần thiết:

```bash
pip install aiohttp beautifulsoup4 python-dotenv
```

#### **4. Chạy Script:**

```bash
python batch_script_async.py
```

#### **5. Lưu Ý:**

- **Giới hạn Tỷ lệ Yêu cầu**: Mặc dù bạn sử dụng async, hãy kiểm soát số lượng kết nối đồng thời (`max_workers`) để tránh quá tải API hoặc hệ thống.

- **Xử lý Lỗi**: Script đã bao gồm cơ chế xử lý lỗi cơ bản. Bạn có thể mở rộng thêm để xử lý các trường hợp phức tạp hơn.

---

## **Tóm Tắt và Khuyến Nghị**

- **Nếu bạn cần xử lý một lượng video lớn một cách thường xuyên và muốn tăng hiệu suất**, chuyển đổi API sang một script cục bộ sử dụng `ThreadPoolExecutor` hoặc `asyncio` là lựa chọn tốt hơn.

- **Nếu bạn muốn duy trì tính linh hoạt và mở rộng để tích hợp vào các ứng dụng khác**, sử dụng API hiện tại với các cải tiến như xử lý song song thông qua nhiều yêu cầu HTTP là phù hợp hơn.

- **Xem xét bảo mật và quản lý API Key**: Nếu bạn chuyển đổi sang script cục bộ, đảm bảo rằng API Key được bảo mật tốt và không bị lộ ra ngoài.

- **Kiểm soát tài nguyên hệ thống**: Khi xử lý hàng loạt video, hãy đảm bảo rằng hệ thống của bạn có đủ tài nguyên (CPU, RAM) để xử lý đồng thời mà không gặp vấn đề về hiệu suất.

- **Xử lý và lưu trữ kết quả**: Dù chọn phương pháp nào, hãy đảm bảo rằng bạn có cơ chế lưu trữ và quản lý kết quả một cách hiệu quả, ví dụ như lưu vào file JSON hoặc cơ sở dữ liệu.

Nếu bạn cần thêm hỗ trợ hoặc gặp phải vấn đề cụ thể nào trong quá trình triển khai, đừng ngần ngại chia sẻ thêm thông tin để mình có thể giúp đỡ bạn một cách chi tiết hơn.

Chúc bạn thành công với dự án của mình!